In [32]:
# "Sanaz Hosseini"- University of North Carolina at Charlotte
# Introduction to Machine Learning Class - Instructor: Prof. Hamed Tabkhi
# Pytorch & Neural Networks
# Homework 5

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
df = pd.read_csv('Housing.csv')

In [3]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = df[num_vars]
df_new.head()

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2


In [5]:
df_new.shape

(545, 6)

In [6]:
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

C:\Users\Sanaz\AppData\Local\Temp\ipykernel_2404\2943118898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [7]:
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

In [8]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [9]:
def model(X1, X2, X3, X4, X5, W1, W2, W3, W4, W5, B):
    return W5*X5 + W4*X4 + W3*X3 + W2*X2 + W1*X1 + B

In [10]:
def loss_fn(Y_p, Y):
    squared_diffs = (Y_p - Y)**2
    return squared_diffs.mean()

In [11]:
W1 = torch.ones(())
W2 = torch.ones(())
W3 = torch.ones(())
W4 = torch.ones(())
W5 = torch.ones(())
B = torch.zeros(())

In [12]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [13]:
train_X = X[train_indices]
train_Y = Y[train_indices]

val_X = X[val_indices]
val_Y = Y[val_indices]

In [14]:
train_X.size()

torch.Size([436, 5])

In [15]:
def training_loop(n_epochs, optimizer, params, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_Y_p = model(train_X[:, 0], train_X[:, 1], train_X[:, 2], train_X[:, 3], train_X[:, 4], *params) 
        train_loss = loss_fn(train_Y_p, train_Y)
                             
        val_Y_p = model(val_X[:, 0], val_X[:, 1], val_X[:, 2], val_X[:, 3], val_X[:, 4], *params) 
        val_loss = loss_fn(val_Y_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
    return params

# Training using Learning Rate = 0.1 and Optimizer = SGD : 

In [17]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0126, Validation loss 0.0084
Epoch 1000, Training loss 0.0124, Validation loss 0.0082
Epoch 1500, Training loss 0.0123, Validation loss 0.0082
Epoch 2000, Training loss 0.0123, Validation loss 0.0082
Epoch 2500, Training loss 0.0123, Validation loss 0.0082
Epoch 3000, Training loss 0.0123, Validation loss 0.0082
Epoch 3500, Training loss 0.0123, Validation loss 0.0082
Epoch 4000, Training loss 0.0123, Validation loss 0.0082
Epoch 4500, Training loss 0.0123, Validation loss 0.0082
Epoch 5000, Training loss 0.0123, Validation loss 0.0082


tensor([0.4299, 0.0736, 0.2643, 0.1371, 0.0974, 0.0447], requires_grad=True)

# Training using Learning Rate = 0.1 and Optimizer = Adam : 

In [19]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0123, Validation loss 0.0082
Epoch 1000, Training loss 0.0123, Validation loss 0.0082
Epoch 1500, Training loss 0.0123, Validation loss 0.0082
Epoch 2000, Training loss 0.0123, Validation loss 0.0082
Epoch 2500, Training loss 0.0123, Validation loss 0.0082
Epoch 3000, Training loss 0.0123, Validation loss 0.0082
Epoch 3500, Training loss 0.0123, Validation loss 0.0082
Epoch 4000, Training loss 0.0123, Validation loss 0.0082
Epoch 4500, Training loss 0.0123, Validation loss 0.0082
Epoch 5000, Training loss 0.0123, Validation loss 0.0082


tensor([0.4299, 0.0736, 0.2643, 0.1371, 0.0974, 0.0447], requires_grad=True)

# Training using Learning Rate = 0.01 and Optimizer = SGD : 

In [21]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0503, Validation loss 0.0349
Epoch 1000, Training loss 0.0231, Validation loss 0.0142
Epoch 1500, Training loss 0.0167, Validation loss 0.0101
Epoch 2000, Training loss 0.0147, Validation loss 0.0092
Epoch 2500, Training loss 0.0138, Validation loss 0.0089
Epoch 3000, Training loss 0.0133, Validation loss 0.0087
Epoch 3500, Training loss 0.0130, Validation loss 0.0086
Epoch 4000, Training loss 0.0128, Validation loss 0.0085
Epoch 4500, Training loss 0.0127, Validation loss 0.0085
Epoch 5000, Training loss 0.0126, Validation loss 0.0084


tensor([0.4682, 0.1778, 0.2504, 0.1149, 0.0841, 0.0042], requires_grad=True)

# Training using Learning Rate = 0.01 and Optimizer = Adam : 

In [23]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.0144, Validation loss 0.0104
Epoch 1000, Training loss 0.0125, Validation loss 0.0084
Epoch 1500, Training loss 0.0123, Validation loss 0.0082
Epoch 2000, Training loss 0.0123, Validation loss 0.0082
Epoch 2500, Training loss 0.0123, Validation loss 0.0082
Epoch 3000, Training loss 0.0123, Validation loss 0.0082
Epoch 3500, Training loss 0.0123, Validation loss 0.0082
Epoch 4000, Training loss 0.0123, Validation loss 0.0082
Epoch 4500, Training loss 0.0123, Validation loss 0.0082
Epoch 5000, Training loss 0.0123, Validation loss 0.0082


tensor([0.4299, 0.0736, 0.2643, 0.1371, 0.0974, 0.0447], requires_grad=True)

# Training using Learning Rate = 0.001 and Optimizer = SGD : 

In [25]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.2282, Validation loss 0.1348
Epoch 1000, Training loss 0.1416, Validation loss 0.0973
Epoch 1500, Training loss 0.1196, Validation loss 0.0881
Epoch 2000, Training loss 0.1041, Validation loss 0.0776
Epoch 2500, Training loss 0.0912, Validation loss 0.0676
Epoch 3000, Training loss 0.0802, Validation loss 0.0587
Epoch 3500, Training loss 0.0708, Validation loss 0.0512
Epoch 4000, Training loss 0.0628, Validation loss 0.0448
Epoch 4500, Training loss 0.0560, Validation loss 0.0394
Epoch 5000, Training loss 0.0502, Validation loss 0.0348


tensor([ 0.6948,  0.5598,  0.6356,  0.3312,  0.3560, -0.3907],
       requires_grad=True)

# Training using Learning Rate = 0.001 and Optimizer = Adam : 

In [27]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 0.1364, Validation loss 0.0789
Epoch 1000, Training loss 0.0543, Validation loss 0.0395
Epoch 1500, Training loss 0.0365, Validation loss 0.0271
Epoch 2000, Training loss 0.0248, Validation loss 0.0179
Epoch 2500, Training loss 0.0186, Validation loss 0.0133
Epoch 3000, Training loss 0.0158, Validation loss 0.0114
Epoch 3500, Training loss 0.0145, Validation loss 0.0105
Epoch 4000, Training loss 0.0137, Validation loss 0.0097
Epoch 4500, Training loss 0.0132, Validation loss 0.0091
Epoch 5000, Training loss 0.0128, Validation loss 0.0087


tensor([ 0.4549,  0.2265,  0.2172,  0.1177,  0.0913, -0.0122],
       requires_grad=True)

# Training using Learning Rate = 0.0001 and Optimizer = SGD : 

In [29]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 1.0259, Validation loss 0.7296
Epoch 1000, Training loss 0.8186, Validation loss 0.5640
Epoch 1500, Training loss 0.6609, Validation loss 0.4408
Epoch 2000, Training loss 0.5408, Validation loss 0.3495
Epoch 2500, Training loss 0.4492, Validation loss 0.2820
Epoch 3000, Training loss 0.3792, Validation loss 0.2321
Epoch 3500, Training loss 0.3256, Validation loss 0.1955
Epoch 4000, Training loss 0.2844, Validation loss 0.1686
Epoch 4500, Training loss 0.2527, Validation loss 0.1490
Epoch 5000, Training loss 0.2281, Validation loss 0.1348


tensor([ 0.8414,  0.7522,  0.9005,  0.7620,  0.7872, -0.5162],
       requires_grad=True)

# Training using Learning Rate = 0.0001 and Optimizer = Adam : 

In [31]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)

Epoch 500, Training loss 1.0572, Validation loss 0.7621
Epoch 1000, Training loss 0.8524, Validation loss 0.6026
Epoch 1500, Training loss 0.6794, Validation loss 0.4697
Epoch 2000, Training loss 0.5346, Validation loss 0.3600
Epoch 2500, Training loss 0.4149, Validation loss 0.2711
Epoch 3000, Training loss 0.3177, Validation loss 0.2007
Epoch 3500, Training loss 0.2406, Validation loss 0.1466
Epoch 4000, Training loss 0.1814, Validation loss 0.1070
Epoch 4500, Training loss 0.1377, Validation loss 0.0796
Epoch 5000, Training loss 0.1069, Validation loss 0.0622


tensor([ 0.6132,  0.6130,  0.5900,  0.5952,  0.5932, -0.3725],
       requires_grad=True)